In [1]:
import findspark
findspark.init('/home/harkirat/spark-3.3.1-bin-hadoop3')

import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('lreg').getOrCreate()

22/12/26 14:47:35 WARN Utils: Your hostname, harkirat-QEMU-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 192.168.64.16 instead (on interface enp0s6)
22/12/26 14:47:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/26 14:47:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark=SparkSession.builder.appName('logreg').getOrCreate()

22/12/26 14:48:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df=spark.read.format('libsvm').load('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/sample_libsvm_data.txt')

22/12/26 14:51:58 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
log_reg=LogisticRegression()

In [9]:
fitted_log_reg=log_reg.fit(df)

22/12/26 14:54:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/26 14:54:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [10]:
results=fitted_log_reg.summary

In [13]:
results.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [14]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514870...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198872...|[6.76550379997544...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234669...|[4.87842678719363...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012715...|[4.62137287300564...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874693...|[1.81823629114105...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504180...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212819...|[6.97903542820426...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446273...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606545...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [15]:
train, test = df.randomSplit([0.7, 0.3])

In [16]:
final_model=LogisticRegression()

In [17]:
fit_final=final_model.fit(train)

In [18]:
predictions_and_labels=fit_final.evaluate(test)

In [19]:
predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[22.7398545406120...|[0.99999999986689...|       0.0|
|  0.0|(692,[100,101,102...|[0.80641604351069...|[0.69134525952484...|       0.0|
|  0.0|(692,[122,123,124...|[23.1914410475673...|[0.99999999991526...|       0.0|
|  0.0|(692,[123,124,125...|[35.3975689628466...|[0.99999999999999...|       0.0|
|  0.0|(692,[125,126,127...|[34.6232413868969...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[17.0388432887613...|[0.99999996017788...|       0.0|
|  0.0|(692,[126,127,128...|[37.4473682897191...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[26.0113634679192...|[0.99999999999494...|       0.0|
|  0.0|(692,[127,128,129...|[22.8257552627881...|[0.99999999987784...|       0.0|
|  0.0|(692,[127

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [21]:
my_eval=BinaryClassificationEvaluator()

In [22]:
roc=my_eval.evaluate(predictions_and_labels.predictions)

In [23]:
roc

1.0

# TITANIC DATASET

In [25]:
spark=SparkSession.builder.appName('titanic').getOrCreate()

22/12/26 15:49:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [28]:
df = spark.read.csv('Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv',inferSchema=True,header=True)

In [29]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [30]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [32]:
my_cols=df.select([
     'Survived',
     'Pclass',
     'Sex',
     'Age',
     'SibSp',
     'Parch',
     'Fare',
     'Embarked'
])

In [33]:
final_df=my_cols.na.drop()

## Categorical Columns

In [34]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [35]:
gender_indexer=StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder=OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [36]:
embark_indexer=StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder=OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [37]:
assembler=VectorAssembler(inputCols=['Pclass',
                                     'SexVec',
                                     'Age',
                                     'SibSp',
                                     'Parch',
                                     'Fare',
                                     'EmbarkVec'],outputCol='features')

## Pipeline

In [38]:
from pyspark.ml import Pipeline

In [40]:
log_reg=LogisticRegression(featuresCol='features', labelCol='Survived')

In [42]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_encoder,embark_encoder,
                            assembler,log_reg])

In [43]:
train, test=final_df.randomSplit([0.7,0.3])

In [44]:
fit_model=pipeline.fit(train)

In [45]:
results=fit_model.transform(test)

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [47]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                        labelCol='Survived')

In [48]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [49]:
AUC = my_eval.evaluate(results)
AUC

0.7986452082196763